# Nannipieri et al. (1977)

## DEB model calibration

### Importing functions

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

### This is the DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3];
    
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    #k=pars[5];
    ce=pars[5];
    MX1=ce/4;
    
    #Scaling function for substrate uptake
    f=S/(Km+S)
    #Fluxes
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e + g)
    
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = growth*X1 #- k*X1
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1 
           
    return dSdt, dedt, dX1dt, dCO2dt;

### The function below uses the output from DEBmodel to convert biomass pools $e$ and $X_{1}$ to microbial biomass ($B$) and biomass proxy-parameter ATP. 

Both, $B$ and ATP are in C1-mols. However, the biomass of microbes in soil was measured as a dry weight of all microbes extracted from soil denoted below by symbol $W$. Two additional parameters converting C1-mols to total mass of $X_{1}$ and $e$ are defined, $M_{W_{X_{1}}}$ and $M_{W_{e}}$ respectively.

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, tX1, te, MwX1, Mwe
    conversions=pars[5:10]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and total ATP
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    #Flush = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])*B
    ATP = (conversions[1]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
    W = (conversions[3]*conversions[0]/4 + conversions[4]*conversions[0]*y[:, 1])*y[:, 2]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           W.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

### Objective function is defined

In [4]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, tX1, te, MwX1, Mwe
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    e_i = 0.25*(pars[6] - d.ATPinit[0]/d.Winit[0]*pars[8])/(d.ATPinit[0]/d.Winit[0]*(pars[9] - pars[7]))
    X1_i = d.Winit[0]/(pars[5]*(0.25*pars[8] + pars[9]*e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
     
    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

### Goodness of fit is calculated

In [5]:
def goodnessDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, tX1, te, MwX1, Mwe
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    e_i = 0.25*(pars[6] - d.ATPinit[0]/d.Winit[0]*pars[8])/(d.ATPinit[0]/d.Winit[0]*(pars[9] - pars[7]))
    X1_i = d.Winit[0]/(pars[5]*(0.25*pars[8] + pars[9]*e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, tX1, te, MwX1, Mwe
    conversions=pars[5:10]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and total ATP
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    #Flush = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])*B
    ATP = (conversions[1]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
    W = (conversions[3]*conversions[0]/4 + conversions[4]*conversions[0]*y[:, 1])*y[:, 2]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 3.60, 0.05)),1),#glucose
                           y[:, 1].reshape(len(np.arange(0, 3.60, 0.05)),1),#e
                           y[:, 2].reshape(len(np.arange(0, 3.60, 0.05)),1),#X1
                           W.reshape(len(np.arange(0, 3.60, 0.05)),1),#biomass in ug/g
                           B.reshape(len(np.arange(0, 3.60, 0.05)),1),#biomass in umols(C)/g
                           y[:, 3].reshape(len(np.arange(0, 3.60, 0.05)),1),#CO2
                           #Flush.reshape(len(np.arange(0, 8.05, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 3.60, 0.05)),1)), axis=1)

    return yhat

## Treatment A

In [7]:
#reading the data
dall = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Nannipieri1977.csv', sep=',')
#select treatment A
d = dall[dall.Treatment == "A"]
print(d)

                      Study       Soil Substrate  Clay  pH  Ctot  Ntot  \
0  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
1  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
2  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
3  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
4  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
5  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
6  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
7  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   

       Time       Winit   ATPinit      Sinit            W       ATP  \
0  0.000000  824.322727  0.018547  249.79184   824.322727  0.018547   
1  0.250000  824.322727  0.018547  249.79184   837.861273  0.024626   
2  0.500000  824.322727  0.018547  249.79184  1137.019455  0.075747   
3  1.000000  824.322727  0.018547  249.79184  114

In [8]:
n_debparsA=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe
                                             #(0, 1)]) #iX1

<ipython-input-4-09ab2a41bc05>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.Winit[0]/(pars[5]*(0.25*pars[8] + pars[9]*e_i))
<ipython-input-2-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-107b63ff77bc>:15: RuntimeWarning: invalid value encountered in multiply
  ATP = (conversions[1]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
<ipython-input-3-107b63ff77bc>:16: RuntimeWarning: invalid value encountered in multiply
  W = (conversions[3]*conversions[0]/4 + conversions[4]*conversions[0]*y[:, 1])*y[:, 2]
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (p

In [9]:
print(n_debparsA)
print(goodnessDEB(n_debparsA.x))

     fun: 6.502436990732981
 message: ['Maximum number of iteration reached']
    nfev: 79313
    nhev: 0
     nit: 1000
    njev: 5392
  status: 0
 success: True
       x: array([5.00000000e-02, 1.42505176e+03, 3.97317669e+00, 1.00000000e-01,
       1.91361889e+00, 1.72591343e+00, 1.71096136e-02, 2.12417902e-01,
       7.96361760e+02, 2.98420984e+02])
[ 0.98904397 -0.12599434 20.25198867  6.91722989]


In [10]:
n_debparsDEA=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe

In [11]:
print(n_debparsDEA)
print(goodnessDEB(n_debparsDEA.x))

     fun: 3.4979828894891547
     jac: array([ 1.17788446e-02,  5.66575006e-04,  7.75813188e-02,  2.04702921e-02,
       -4.53379559e-03, -1.94955147e-05, -1.39632084e-01,  5.32924815e-03,
       -9.96002490e-04,  7.43493565e-04])
 message: 'Optimization terminated successfully.'
    nfev: 47639
     nit: 313
 success: True
       x: array([5.07036384e-02, 1.04924814e+03, 7.23636776e-01, 9.42199189e-02,
       2.06965160e+00, 8.44908449e+00, 2.67718195e-03, 6.04368151e-02,
       1.28335792e+02, 1.77289307e+02])
[ 0.99139387 -0.09897052 20.19794105  3.67133953]


## Solution over time for visualization in R

In [12]:
#initial conditions
S_i = d.Sinit[0]
    
e_i = 0.25*(n_debparsDEA.x[6] - d.ATPinit[0]/d.Winit[0]*n_debparsDEA.x[8])/(d.ATPinit[0]/d.Winit[0]*(n_debparsDEA.x[9] - n_debparsDEA.x[7]))
X1_i = d.Winit[0]/(n_debparsDEA.x[5]*(0.25*n_debparsDEA.x[8] + n_debparsDEA.x[9]*e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredA = predDEB(DEBmodel, n_debparsDEA.x, t, y0)
   
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002Pred.csv', Joergensen2002Pred, delimiter=",")

## Treatment B

In [13]:
#select treatment B
d = dall[dall.Treatment == "B"]
d = d.reset_index()
print(d)

   index                     Study       Soil Substrate  Clay  pH  Ctot  Ntot  \
0      8  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
1      9  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
2     10  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
3     11  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
4     12  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
5     13  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
6     14  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
7     15  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   

       Time       Winit   ATPinit      Sinit            W       ATP  \
0  0.000000  835.240909  0.019872  249.79184   835.240909  0.019872   
1  0.250000  835.240909  0.019872  249.79184   778.248000  0.025421   
2  0.500000  835.240909  0.019872  249.79184  1031.986545

In [14]:
n_debparsB=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe
                                             #(0, 1)]) #iX1

/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-4-09ab2a41bc05>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.Winit[0]/(pars[5]*(0.25*pars[8] + pars[9]*e_i))
<ipython-input-2-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-107b63ff77bc>:15: RuntimeWarning: invalid value encountered in multiply
  ATP = (conversions[1]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
/usr/local/lib/python3.8/dist-packages/scipy/optim

In [15]:
print(n_debparsB)
print(goodnessDEB(n_debparsB.x))

     fun: 4.340764335686849
 message: ['Maximum number of iteration reached']
    nfev: 84450
    nhev: 0
     nit: 1000
    njev: 5859
  status: 0
 success: True
       x: array([5.00000000e-02, 6.48748016e+02, 2.46677329e+00, 1.00000000e-01,
       1.88247573e+00, 4.07676929e+00, 1.73422186e-02, 8.87152897e-02,
       7.51574283e+02, 1.05758500e+02])
[ 0.98954258 -0.12026028 20.24052055  4.76272824]


In [16]:
n_debparsDEB=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe

In [17]:
print(n_debparsDEB)
print(goodnessDEB(n_debparsDEB.x))

     fun: 4.123320670995511
     jac: array([ 1.18571819e-03, -3.47281435e-05,  4.87013099e-02, -3.27119665e-02,
        9.29533779e-02,  8.88178420e-08, -8.90489460e-02, -2.05382378e-03,
        1.73017019e-04, -3.34132931e-04])
 message: 'Optimization terminated successfully.'
    nfev: 35609
     nit: 235
 success: True
       x: array([5.01248920e-02, 1.31228922e+03, 2.00985028e+00, 7.72345547e-02,
       9.36798750e-01, 3.48519680e-01, 7.45920583e-03, 7.36924214e-02,
       3.23836566e+02, 9.67104088e+01])
[ 0.98940097 -0.1218889  20.2437778   4.45023757]


## Solution over time for visualization in R

In [18]:
#initial conditions
S_i = d.Sinit[0]
    
e_i = 0.25*(n_debparsDEB.x[6] - d.ATPinit[0]/d.Winit[0]*n_debparsDEB.x[8])/(d.ATPinit[0]/d.Winit[0]*(n_debparsDEB.x[9] - n_debparsDEB.x[7]))
X1_i = d.Winit[0]/(n_debparsDEB.x[5]*(0.25*n_debparsDEB.x[8] + n_debparsDEB.x[9]*e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredB = predDEB(DEBmodel, n_debparsDEB.x, t, y0)
   
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002Pred.csv', Joergensen2002Pred, delimiter=",")

## Treatment C

In [19]:
#select treatment C
d = dall[dall.Treatment == "C"]
d = d.reset_index()
print(d)

   index                     Study       Soil Substrate  Clay  pH  Ctot  Ntot  \
0     16  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
1     17  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
2     18  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
3     19  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
4     20  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
5     21  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
6     22  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   
7     23  Nannipieri et al. (1977)  Chernozem   Glucose   NaN   7     2   NaN   

       Time       Winit   ATPinit      Sinit            W       ATP  \
0  0.000000  739.597636  0.022787  124.89592   739.597636  0.022787   
1  0.250000  739.597636  0.022787  124.89592   728.024364  0.020652   
2  0.500000  739.597636  0.022787  124.89592   897.037818

In [20]:
n_debparsC=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe
                                             #(0, 1)]) #iX1

/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-4-09ab2a41bc05>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.Winit[0]/(pars[5]*(0.25*pars[8] + pars[9]*e_i))
<ipython-input-2-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-107b63ff77bc>:15: RuntimeWarning: invalid value encountered in multiply
  ATP = (conversions[1]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
<ipython-input-3-107b63ff77bc>:16: RuntimeWarning:

In [21]:
print(n_debparsC)
print(goodnessDEB(n_debparsC.x))

     fun: 6.494172413620179
 message: ['Maximum number of iteration reached']
    nfev: 85605
    nhev: 0
     nit: 1000
    njev: 5964
  status: 0
 success: True
       x: array([5.00000000e-02, 9.77543610e+02, 3.15857670e+00, 1.00000000e-01,
       1.81289903e+00, 5.42793407e+00, 2.11495622e-02, 1.67901705e-01,
       7.21649803e+02, 3.61639045e+02])
[ 0.98717446 -0.14108093 20.28216186  7.22648289]


In [22]:
n_debparsDEC=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             #(0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1), #te
                                              (10, 1000), #MwX1
                                              (10, 1000)]) #Mwe

In [23]:
print(n_debparsDEC)
print(goodnessDEB(n_debparsDEC.x))

     fun: 3.822039526457745
     jac: array([ 2.29336550e-03,  2.66586541e-04,  1.92339477e-03, -8.86403218e-01,
       -9.26174698e-03,  4.50306462e-04,  4.55817606e-03,  2.39617215e-03,
        1.48276829e-03, -1.06092828e-03])
 message: 'Optimization terminated successfully.'
    nfev: 30960
     nit: 201
 success: True
       x: array([5.06900406e-02, 5.77767647e+02, 8.64982089e-01, 1.00000000e-01,
       3.97342740e+00, 3.10156123e+00, 6.59509569e-03, 6.64662763e-02,
       2.27128513e+02, 2.32426674e+02])
[ 0.99035503 -0.10609462 20.21218924  4.03041223]


## Solution over time for visualization in R

In [24]:
#initial conditions
S_i = d.Sinit[0]
    
e_i = 0.25*(n_debparsDEC.x[6] - d.ATPinit[0]/d.Winit[0]*n_debparsDEC.x[8])/(d.ATPinit[0]/d.Winit[0]*(n_debparsDEC.x[9] - n_debparsDEC.x[7]))
X1_i = d.Winit[0]/(n_debparsDEC.x[5]*(0.25*n_debparsDEC.x[8] + n_debparsDEC.x[9]*e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredC = predDEB(DEBmodel, n_debparsDEC.x, t, y0)

Nannipieri1977Pred = np.concatenate([nPredA, nPredB, nPredC])
   
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Nannipieri1977Pred.csv', Nannipieri1977Pred, delimiter=",")

In [25]:
Nannipieri1977Pars = np.concatenate([n_debparsDEA.x, n_debparsDEB.x, n_debparsDEC.x])

In [26]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Nannipieri1977Pars.csv', Nannipieri1977Pars.reshape(3,10), delimiter=",")

# Monod and Pirt models are fitted for comparison to DEB

## Monod model
Model as well as supplementary functions are defined below

In [27]:
def Mmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    k=pars[3];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE)
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [28]:
def calcM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##katp, Mw
    conversions=pars[4:6]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    ATP = conversions[0]*y[:, 1]
    W = conversions[1]*y[:, 1]    
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time),1),#CO2
                           W.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

In [29]:
def obj_funM (x):
    #define parameters
    ##v, Km, CUE, k, katp, Mw
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
       
    B_i = d.ATPinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
    
    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [30]:
def goodnessM (x):
    #define parameters
    ##v, Km, CUE, k, katp, Mw
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.ATPinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
        
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                          np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [31]:
def predM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
     ##katp, Mw
    conversions=pars[4:6]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    ATP = conversions[0]*y[:, 1]
    W = conversions[1]*y[:, 1]  
    
    yhat = np.concatenate((#y[:, 0].reshape(len(np.arange(0, 3.60, 0.05)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 3.60, 0.05)),1),#CO2
                           W.reshape(len(np.arange(0, 3.60, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 3.60, 0.05)),1)), axis=1)

    return yhat

## Treatment A

In [32]:
d = dall[dall.Treatment == "A"]

In [33]:
n_mparsA=differential_evolution(obj_funM, [(0.0001, 20), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

In [34]:
print(n_mparsA)
print(goodnessM(n_mparsA.x))

     fun: 22.047869658517172
     jac: array([-5.06172840e-02,  1.05481185e-03, -4.39725021e+00,  4.93681682e+00,
        4.17034727e+00, -8.01556088e-02])
 message: 'Optimization terminated successfully.'
    nfev: 17246
     nit: 190
 success: True
       x: array([1.85548360e+01, 1.53355843e+03, 4.27596255e-01, 3.27250529e-01,
       4.02998118e-04, 1.53969065e+01])
[ 0.98890016 -0.12764816 12.25529633  2.7167664 ]


In [35]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_mparsA.x[4]
    
y0 = np.array([S_i, B_i, 0])
#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredMA = predM(Mmodel, n_mparsA.x, t, y0)
    
#model simulations
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredM.csv', Joergensen2002PredM, delimiter=",")

## Treatment B

In [36]:
d = dall[dall.Treatment == "B"]
d = d.reset_index()

In [37]:
n_mparsB=differential_evolution(obj_funM, [(0.0001, 20), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

In [38]:
print(n_mparsB)
print(goodnessM(n_mparsB.x))

     fun: 20.289556404091947
     jac: array([-5.61115552e-03,  1.08003638e-04, -3.12043014e+00,  3.43142936e+00,
        2.57999417e+00, -6.93830713e-02])
 message: 'Optimization terminated successfully.'
    nfev: 16256
     nit: 179
 success: True
       x: array([1.91483836e+01, 1.76362514e+03, 3.95845184e-01, 1.92254159e-01,
       4.28325110e-04, 1.36278917e+01])
[ 0.97687532 -0.26593382 12.53186764  5.10895749]


In [39]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_mparsB.x[4]
    
y0 = np.array([S_i, B_i, 0])
#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredMB = predM(Mmodel, n_mparsB.x, t, y0)
    
#model simulations
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredM.csv', Joergensen2002PredM, delimiter=",")

## Treatment C

In [40]:
d = dall[dall.Treatment == "C"]
d = d.reset_index()

In [41]:
n_mparsC=differential_evolution(obj_funM, [(0.0001, 20), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

<ipython-input-29-bb41636d0079>:9: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.ATPinit[0]/pars[4]
<ipython-input-27-15e34bf3de91>:14: RuntimeWarning: invalid value encountered in double_scalars
  respiration = uptake*(1 - CUE)
<ipython-input-27-15e34bf3de91>:19: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death
<ipython-input-28-b5e55aa73bc0>:14: RuntimeWarning: invalid value encountered in multiply
  ATP = conversions[0]*y[:, 1]


In [42]:
print(n_mparsC)
print(goodnessM(n_mparsC.x))

     fun: 46.266906940140906
 message: 'Optimization terminated successfully.'
    nfev: 5691
     nit: 62
 success: True
       x: array([1.15725973e+01, 3.04058737e+01, 1.43182363e-02, 2.72703097e-02,
       2.91464358e-01, 9.90286050e+03])
[ 0.97267688 -0.30055435 12.60110869 15.76248771]


In [43]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_mparsC.x[4]
    
y0 = np.array([S_i, B_i, 0])
#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredMC = predM(Mmodel, n_mparsC.x, t, y0)
    
#model simulations
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredM.csv', Joergensen2002PredM, delimiter=",")

In [44]:
Nannipieri1977PredM = np.concatenate([nPredMA, nPredMB, nPredMC])
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Nannipieri1977PredM.csv', Nannipieri1977PredM, delimiter=",")

## Pirt model
Model as well as supplementary functions are defined below

In [45]:
def Pmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    m = pars[3];
    k = pars[4];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE) + B*m
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [46]:
def calcP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##katp, MW
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    ATP = conversions[0]*y[:, 1]
    W = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time[0:18]),1),#CO2
                           W.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

In [47]:
def obj_funP (x):
    #define parameters
    ##v, Km, CUE, m, k, katp, Mw
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.ATPinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
        
    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

     #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
       
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [48]:
def goodnessP (x):
    #define parameters
    ##v, Km, CUE, k, katp, MW
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.ATPinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        np.array([d.W]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    ##std    
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.W)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.W).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [49]:
def predP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##katp, Mw
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    ATP = conversions[0]*y[:, 1]
    W = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(np.arange(0, 86, 1)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 3.60, 0.05)),1),#CO2
                           W.reshape(len(np.arange(0, 3.60, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 3.60, 0.05)),1)), axis=1)

    return yhat

## Treatment A

In [50]:
d = dall[dall.Treatment == "A"]

In [51]:
n_pparsA=differential_evolution(obj_funP, [(0.001, 500), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (1e-12, 0.5), #m
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

<ipython-input-47-21b11741bcee>:9: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.ATPinit[0]/pars[5]
<ipython-input-45-c3d2819ce9aa>:14: RuntimeWarning: invalid value encountered in double_scalars
  growth = uptake*CUE
<ipython-input-46-e64599a10017>:13: RuntimeWarning: invalid value encountered in multiply
  ATP = conversions[0]*y[:, 1]
<ipython-input-45-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death


In [52]:
print(n_pparsA)
print(goodnessP(n_pparsA.x))

     fun: 21.944400290467044
 message: 'Optimization terminated successfully.'
    nfev: 21339
     nit: 202
 success: True
       x: array([8.27037127e+01, 7.62607859e+03, 4.70702331e-01, 2.49940703e-02,
       2.81806781e-01, 3.68023092e-04, 1.30475023e+01])
[ 0.98444882 -0.17883855 14.35767711  9.73414206]


In [53]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_pparsA.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredPA = predP(Pmodel, n_pparsA.x, t, y0)
    
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredP.csv', Joergensen2002PredP, delimiter=",")

## Treatment B

In [54]:
d = dall[dall.Treatment == "B"]
d = d.reset_index()

In [55]:
n_pparsB=differential_evolution(obj_funP, [(0.001, 500), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (1e-12, 0.5), #m
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

<ipython-input-47-21b11741bcee>:9: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.ATPinit[0]/pars[5]
<ipython-input-45-c3d2819ce9aa>:15: RuntimeWarning: invalid value encountered in double_scalars
  respiration = uptake*(1 - CUE) + B*m
<ipython-input-45-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death
<ipython-input-46-e64599a10017>:13: RuntimeWarning: invalid value encountered in multiply
  ATP = conversions[0]*y[:, 1]


In [56]:
print(n_pparsB)
print(goodnessP(n_pparsB.x))

     fun: 21.007982567204277
 message: 'Optimization terminated successfully.'
    nfev: 18294
     nit: 173
 success: True
       x: array([4.83292697e+02, 1.11016333e+03, 1.09070117e-02, 2.20923208e-01,
       4.86875219e-01, 3.68810853e-02, 1.30378134e+03])
[ 0.98436688 -0.17978088 14.35956175 10.25974141]


In [57]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_pparsB.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredPB = predP(Pmodel, n_pparsB.x, t, y0)
    
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredP.csv', Joergensen2002PredP, delimiter=",")

## Treatment C

In [58]:
d = dall[dall.Treatment == "C"]
d = d.reset_index()

In [59]:
n_pparsC=differential_evolution(obj_funP, [(0.001, 500), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (1e-12, 0.5), #m
                                             (0, 1), #katp
                                             (10, 10000)]) #Mw
                                             #(0, 1)]) #iX1

<ipython-input-47-21b11741bcee>:9: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.ATPinit[0]/pars[5]
<ipython-input-45-c3d2819ce9aa>:15: RuntimeWarning: invalid value encountered in double_scalars
  respiration = uptake*(1 - CUE) + B*m
<ipython-input-45-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death
<ipython-input-46-e64599a10017>:13: RuntimeWarning: invalid value encountered in multiply
  ATP = conversions[0]*y[:, 1]


In [60]:
print(n_pparsC)
print(goodnessP(n_pparsC.x))

     fun: 18.77013530808327
 message: 'Optimization terminated successfully.'
    nfev: 21759
     nit: 206
 success: True
       x: array([4.52650753e+02, 5.11477284e+03, 9.38181302e-02, 1.71810539e-03,
       4.98721118e-01, 6.15573374e-03, 1.81169478e+02])
[ 0.98955555 -0.11488896 14.22977792 10.60280896]


In [61]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.ATPinit[0]/n_pparsC.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 3.60, 0.05)

#model simulations
nPredPC = predP(Pmodel, n_pparsC.x, t, y0)
    
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredP.csv', Joergensen2002PredP, delimiter=",")

In [62]:
Nannipieri1977PredP = np.concatenate([nPredPA, nPredPB, nPredPC])

In [63]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Nannipieri1977PredP.csv', Nannipieri1977PredP, delimiter=",")